In [7]:
#importing needed packages and files
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
file_path = '/content/drive/MyDrive/SMSstuff/Churn_Modelling.csv'
print(file_path)
import numpy as np
import pandas as pd


Mounted at /content/drive
/content/drive/MyDrive/SMSstuff/Churn_Modelling.csv


In [8]:
df = pd.read_csv(file_path)
df.head()



,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


#Preprocessing

In [9]:
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
df.head()




,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [10]:
#checking for nulls
null = pd.DataFrame(df.isnull().sum())
null




,0
CreditScore,0
Geography,0
Gender,0
Age,0
Tenure,0
Balance,0
NumOfProducts,0
HasCrCard,0
IsActiveMember,0
EstimatedSalary,0


In [11]:
#checking for any connection with exited for model
df.corrwith(df.Exited).drop(columns = ['Exited']).sort_values(ascending=False)*100


<ipython-input-11-9c93a31eed5f>:2: FutureWarning: The default value of numeric_only in DataFrame.corrwith is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corrwith(df.Exited).drop(columns = ['Exited']).sort_values(ascending=False)*100


Exited             100.000000
Age                 28.532304
Balance             11.853277
EstimatedSalary      1.209686
HasCrCard           -0.713777
Tenure              -1.400061
CreditScore         -2.709354
NumOfProducts       -4.781986
IsActiveMember     -15.612828
dtype: float64

In [12]:
gender = {'Female': 0, 'Male': 1}
df['Gender'] = df['Gender'].map(gender)



In [15]:
#geographical encoding
geography = {'France': 0, 'Germany': 1, 'Spain': 2}
df['Geography'] = df['Geography'].map(geography)


Age and is active member correlate as they play a role

#Scaling the dataset and Train Test Split

In [18]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()

In [14]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=['Exited']).values, df.Exited.values,
                                                    test_size=0.2, random_state=42)

In [19]:
x_train = ss.fit_transform(x_train)
x_test = ss.transform(x_test)



##Training a model

##1. Random Forest

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Initialize and fit the Random Forest classifier on the training data
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(x_train, y_train)

# Make predictions on the testing data
y_pred = clf.predict(x_test)

# Generate a classification report
report = classification_report(y_test, y_pred)

# Print the classification report
print(report)

              precision    recall  f1-score   support

           0       0.88      0.96      0.92      1607
           1       0.75      0.47      0.58       393

    accuracy                           0.86      2000
   macro avg       0.81      0.71      0.75      2000
weighted avg       0.85      0.86      0.85      2000



##2. Decision Tree

In [22]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

# Initialize and fit the Decision Tree classifier on the training data
clf = DecisionTreeClassifier(random_state=42)
clf.fit(x_train, y_train)

# Make predictions on the testing data
y_pred = clf.predict(x_test)

# Generate a classification report
report = classification_report(y_test, y_pred)

# Print the classification report
print(report)

              precision    recall  f1-score   support

           0       0.88      0.84      0.86      1607
           1       0.45      0.52      0.48       393

    accuracy                           0.78      2000
   macro avg       0.66      0.68      0.67      2000
weighted avg       0.79      0.78      0.79      2000



##3. KNN


In [23]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

# Initialize and fit the KNN classifier on the training data
k = 3  # Set the number of neighbors (you can choose an appropriate value)
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(x_train, y_train)

# Make predictions on the testing data
y_pred = knn.predict(x_test)

# Generate a classification report
report = classification_report(y_test, y_pred)

# Print the classification report
print(report)


              precision    recall  f1-score   support

           0       0.87      0.93      0.90      1607
           1       0.62      0.45      0.52       393

    accuracy                           0.84      2000
   macro avg       0.75      0.69      0.71      2000
weighted avg       0.82      0.84      0.83      2000



##4.SVM

In [24]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Initialize and fit the SVM classifier on the training data
svm = SVC(kernel='linear', random_state=42)  # You can choose a kernel type (linear, polynomial, or radial basis function, etc.)
svm.fit(x_train, y_train)

# Make predictions on the testing data
y_pred = svm.predict(x_test)

# Generate a classification report
report = classification_report(y_test, y_pred)

# Print the classification report
print(report)


              precision    recall  f1-score   support

           0       0.80      1.00      0.89      1607
           1       0.00      0.00      0.00       393

    accuracy                           0.80      2000
   macro avg       0.40      0.50      0.45      2000
weighted avg       0.65      0.80      0.72      2000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Therefore using Random Forest was the best as it resulted in the highest accuracy scores overall and with an accuracy of 88%.